In [ ]:
from openai import OpenAI
import json
import time
import pandas as pd
from tqdm import tqdm

# Load data

In [ ]:
data_df = pd.read_csv('..\\result\\merge_df_10.csv')
data_df.columns

In [ ]:
data_df['user_biog'] = 'The biography of ' + data_df['username'] + ' on instagram is ' + data_df['biography'] + '. '
data_df['user_5texts'] = 'The texts of first five posts of ' + data_df['username'] + ' on instagram is ' + data_df['text'] + '. '

# Use ChatGPT to construct kg 

In [ ]:
client = OpenAI(
    api_key="xxx",
)

In [ ]:
prompt_bio = f"""
            You are an AI expert specializing in knowledge graph creation with the goal of capturing relationships based on a given input or request.
            Based on the Instagram user input, including username and biography.
            Your task is to create a knowledge graph based on the input. 
            Output only in json format, including nodes and edges. Make sure the format of json is 100% right.
            Nodes must have a name and an attribute. The name is a direct word or phrase from the input. The attribute is the attribute of the node, please identify the attribute from the following categories: 'username', 'location', 'occupation_or_industry', 'interests', 'self_description', 'brand_collaborations', 'product_category'.
            Among them, the name of product_category is selected from the following list: ['Tools & Home Improvement', 'Automotive', 'Home & Kitchen', 'Toys & Games', 'Patio, Lawn & Garden', 'Sports & Outdoors','Clothing, Shoes & Jewelry', 'Electronics', 'Arts, Crafts & Sewing', 'Industrial & Scientific', 'Pet Supplies', 'Office Products', 'Musical Instruments', 'Grocery & Gourmet Food', 'Beauty & Personal Care', 'Appliances', 'Health & Household', 'Baby Products', 'Cell Phones & Accessories', 'Video Games', 'CDs & Vinyl'].
            Edges must have a relationship, which is a relationship you infer from the input, please identify the relationship from the following categories: 'is_a', 'has_domain_expertise', 'partner', 'target_audience', 'has_brand_association', 'use_product', 'has_interest', 'located_at'.
            If there is @ in the text, it is usually followed by the cooperating brand. If there is # in the texts, it is usually a product category, brand name, character tags or other popular tags. Please make your own inference.
            Respond only with JSON in a format where we can use py2neo/neoj4 package and feed directly into neoj4; to display a graph on the Neo4j browser.
            Make sure the target and source of edges match an existing node. 
            Do not include the markdown triple quotes above and below the JSON, jump straight into it with a curly bracket.
            """


## Use the biography of each user

In [ ]:
json_user_biog = []

for text in tqdm(data_df['user_biog']):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"{text}",
                "response_format": { "type": "json_object" }
            },
            {
                "role": "system",
                "content": prompt_bio,
                "response_format": { "type": "json_object" }
            }
        ],
        model='gpt-3.5-turbo',
        temperature = 0,
    )
    response = chat_completion.choices[0].message.content
    json_user_biog.append(response)

In [ ]:
with open('..\\result\\user_biog.json', 'w') as file:
    json.dump(json_user_biog, file)